# CodeSignal

In [1]:
from collections import defaultdict


class CloudStorageImpl:
    def __init__(self):
        self.files = {}  # {file_name: (size, owner)}
        self.users = {}  # {user_id: {"capacity": int, "used": int}}
        self.user_files = defaultdict(set)  # {user_id: set(file_names)}

        # admin user has unlimited capacity
        self.users["admin"] = {"capacity": float("inf"), "used": 0}

    # -------- Level 1 -------- #
    def add_file(self, name: str, size: int) -> bool:
        return self.add_file_by("admin", name, size) is not None

    def get_file_size(self, name: str) -> int | None:
        return self.files[name][0] if name in self.files else None

    def delete_file(self, name: str) -> int | None:
        if name not in self.files:
            return None

        size, owner = self.files.pop(name)
        if owner != "admin":
            self.users[owner]["used"] -= size
            self.user_files[owner].remove(name)
        return size

    def get_n_largest(self, prefix: str, n: int) -> list[str]:
        filtered = [(f, s) for f, (s, _) in self.files.items() if f.startswith(prefix)]
        if not filtered:
            return []
        filtered.sort(key=lambda x: (-x[1], x[0]))  # size desc, name asc
        return [f"{name}({size})" for name, size in filtered[:n]]

    # -------- Level 2 -------- #
    def add_user(self, user_id: str, capacity: int) -> bool:
        if user_id in self.users:
            return False
        self.users[user_id] = {"capacity": capacity, "used": 0}
        return True

    def add_file_by(self, user_id: str, name: str, size: int) -> int | None:
        if name in self.files or user_id not in self.users:
            return None

        user = self.users[user_id]
        if user["used"] + size > user["capacity"]:
            return None

        # add file
        self.files[name] = (size, user_id)
        self.user_files[user_id].add(name)
        user["used"] += size
        return user["capacity"] - user["used"]

    def merge_user(self, user_id_1: str, user_id_2: str) -> int | None:
        if (
            user_id_1 not in self.users
            or user_id_2 not in self.users
            or user_id_1 == user_id_2
        ):
            return None

        u1, u2 = self.users[user_id_1], self.users[user_id_2]
        u1["capacity"] += u2["capacity"] - u2["used"]

        # transfer ownership of files
        for fname in list(self.user_files[user_id_2]):
            self.files[fname] = (self.files[fname][0], user_id_1)
            self.user_files[user_id_1].add(fname)
        u1["used"] += u2["used"]

        # remove second user
        del self.users[user_id_2]
        del self.user_files[user_id_2]

        return u1["capacity"] - u1["used"]

In [2]:
def test_cloud_storage():
    cs = CloudStorageImpl()

    # Add users
    assert cs.add_user("alice", 100)
    assert cs.add_user("bob", 200)
    assert not cs.add_user("alice", 50)  # duplicate

    # Add files by users
    assert cs.add_file_by("alice", "a.txt", 40) == 60
    assert cs.add_file_by("alice", "b.txt", 60) == 0
    assert cs.add_file_by("alice", "c.txt", 10) is None  # exceeds capacity
    assert cs.add_file_by("bob", "bobfile", 150) == 50

    # File operations
    assert cs.get_file_size("a.txt") == 40
    assert cs.delete_file("a.txt") == 40  # frees Alice’s space
    assert cs.add_file_by("alice", "new.txt", 20) == 20  # ✅ 100 - 60 - 20 = 20

    # Largest files (note: bobby.txt upload fails because Bob lacks capacity)
    assert cs.add_file_by("bob", "bobby.txt", 190) is None
    result = cs.get_n_largest("b", 3)
    assert result == ["bobfile(150)", "b.txt(60)"]

    # Merge users
    remaining = cs.merge_user("alice", "bob")
    assert remaining is not None
    assert "bob" not in cs.users
    assert all(owner != "bob" for _, owner in cs.files.values())

    print("✅ All tests passed!")


if __name__ == "__main__":
    test_cloud_storage()

✅ All tests passed!
